In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')

# Convert timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Sort values to simulate streaming over time
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

# Preview
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [4]:
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficWeight'] = df['TrafficConditionNearby'].map(traffic_map)

# Map vehicle types to weights
vehicle_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

# Sort for streaming logic
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).copy()

In [5]:
# Demand-based pricing model for all parking lots
def demand_price_model(df,
                       alpha=0.6, beta=0.2, gamma=0.3,
                       delta=0.5, epsilon=0.4, lamb=0.5,
                       base_price=10.0):

    all_prices = {}

    for lot_id, group in df.groupby('SystemCodeNumber'):
        group = group.sort_values('Timestamp')
        prices = []

        for _, row in group.iterrows():
            occ = row['Occupancy']
            cap = row['Capacity']
            queue = row['QueueLength']
            traffic = row['TrafficWeight']
            special = row['IsSpecialDay']
            vehicle = row['VehicleWeight']

            # Demand function
            demand = (alpha * (occ / cap) +
                      beta * queue -
                      gamma * traffic +
                      delta * special +
                      epsilon * vehicle)

            # Normalize demand to [0, 1]
            norm_demand = max(0.0, min(demand / 10.0, 1.0))

            # Price update rule
            price = base_price * (1 + lamb * norm_demand)
            price = max(base_price * 0.5, min(price, base_price * 2))

            prices.append(price)

        all_prices[lot_id] = {
            'Timestamp': group['Timestamp'].values,
            'Price': prices
        }

    return all_prices


In [8]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
# Run model 2 and generate results
model2_prices = demand_price_model(df)

# Plot function for any parking lot
def plot_model2(lot_id):
    data = model2_prices[lot_id]
    p = figure(title=f"Model 2 Demand-Based Pricing: {lot_id}",
               x_axis_type='datetime',
               x_axis_label='Time',
               y_axis_label='Price ($)',
               width=800, height=400)

    p.line(data['Timestamp'], data['Price'], color='green', line_width=2, legend_label='Model 2')
    p.legend.location = 'top_left'
    show(p)

# Example usage
plot_model2("BHMBCCMKT01")  # Replace with any SystemCodeNumber


Output hidden; open in https://colab.research.google.com to view.